In [2]:
from PIL import Image, ImageFilter
import random
import numpy as np
import os.path
import copy
import math
import statistics
import matplotlib.pyplot as plt

In [3]:
# FUNCTIONS
# out of bound strokes tileable processing

In [4]:
# left and right
def tileableProcessLR(currStroke, xPos, yPos, sideL, canvas):
    tileXPos = -1*(sideL - xPos)
    canvas.paste(currStroke, (tileXPos,yPos), currStroke)

In [5]:
# top and bottom
def tileableProcessTB(currStroke, xPos, yPos, sideL, canvas):
    tileYPos = -1*(sideL - yPos)
    canvas.paste(currStroke, (xPos,tileYPos), currStroke)
    # after pasting out of bound on top, check if this pasted stroke is out of right bound
    if((currStroke.size[0]+xPos) > sideL):
        tileableProcessLR(currStroke, xPos, tileYPos, sideL, canvas)

In [6]:
# function
# randamly place a currentStroke onto the canvas
def placeStroke(currStroke, canvas, sideL, strokeCoordinates):
    xPos = random.randrange(0, sideL-4)
    yPos = random.randrange(0, sideL-4)
    #print(xPos, yPos)
    
    # find percentage position of the stroke
    xPercent = xPos/sideL
    yPercent = yPos/sideL
    
    strokeCoordinates.append(xPercent) # store relative coordinates (percentage) of stroke
    strokeCoordinates.append(yPercent)
    
    canvas.paste(currStroke, (xPos, yPos), currStroke)
    #canvas.show()
    
    # check if stroke out of canvas, place another one on corresponding position for tilable texture
    if((currStroke.size[0]+xPos) > sideL):
        tileableProcessLR(currStroke, xPos, yPos, sideL, canvas)
    if((currStroke.size[1]+yPos) > sideL):
        tileableProcessTB(currStroke, xPos, yPos, sideL, canvas)
        
    #canvas.show()

In [7]:
# function
# place a currentStroke onto the canvas at a defined specific position
def placeStrokeWithPos(currStroke, canvas, coordinatesToPlace):
    currSideL = canvas.size[0]
    
    # find absolute position for this canvas
    xPos = int(currSideL * coordinatesToPlace[0])
    yPos = int(currSideL * coordinatesToPlace[1])
    
    canvas.paste(currStroke, (xPos, yPos), currStroke)
    #canvas.show()
    
    # check if stroke out of canvas, place another one on corresponding position for tilable texture
    if((currStroke.size[0]+xPos) > sideL):
        tileableProcessLR(currStroke, xPos, yPos, currSideL, canvas)
    if((currStroke.size[1]+yPos) > sideL):
        tileableProcessTB(currStroke, xPos, yPos, currSideL, canvas)
        
    #canvas.show()

In [8]:
# function
# Find tone of the canvas - check the coverage of the canvas
def findCoverage(canvas):
    sideL = canvas.size[0]
    
    # find canvas' coverage rate
    canvasGray = canvas.convert('L')  # turn canvas to grayscale
    #canvasGray.show()
    
    currArray = np.array(canvasGray)  # convert PIL img obj to numpy array
    #print(type(currArray))
    #print(currArray.shape)
    #print(currArray.size)

    #numWhite = np.count_nonzero(narray)
    nonWhite = np.sum(currArray < 255)  # calculate currCanvas nonWhite pixels
    #nonWhite = np.sum(narray > 0)
    
    coverage = nonWhite / (sideL * sideL)
    #print(nonWhite, coverage)
    
    return coverage

In [9]:
# function
# resize a passed in stroke to the same length ratio for next larger texture in the mipmap pyramid
def resizeStrokeForNextLevel(stroke, currStrokeLen, isVertical, bestCanvas, largeCanv):
    """
    if(isVertical == False):  # horizontal stroke
        currLength = currStroke.size[0]
    else:  # vertical stroke
        currLength = currStroke.size[1]
    """
    currLength = currStrokeLen
        
    currSide = bestCanvas.size[0]
    lengthPercent = currLength/currSide
    newLength = int(lengthPercent * largeCanv.size[0])
    
    # resize a stroke for adding into the larger texture canvas
    longStroke = stroke.copy()
    longStroke = longStroke.resize((newLength, stroke.size[1]))
    
    if(isVertical == True):  # vertical stroke
        # rotate the stroke to be vertical direction
        longStroke = longStroke.rotate(rotateDegree, Image.NEAREST, expand = True)
    # if horizontal stroke than just return it
    
    return longStroke

In [10]:
# ************************** API TESTING *********************************
"""
tempTex1 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\16Tone0.1.png")
tempTex2 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\32Tone0.1.png")
tempTex3 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\64Tone0.1.png")
tempTex4 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\128Tone0.1.png")
tempTex5 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\256Tone0.1.png")

tempTexList = []

tempTexList.append(tempTex1)
tempTexList.append(tempTex2)
tempTexList.append(tempTex3)
tempTexList.append(tempTex4)
tempTexList.append(tempTex5)

for i in range(5):
    tempTexList[i].show()

tempTexListCopy = copy.deepcopy(tempTexList)

for i in range(5):
    tempTexListCopy[i].show()
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 27-28: truncated \UXXXXXXXX escape (Temp/ipykernel_26740/1303816656.py, line 24)

In [11]:
# function: returns created canvasCandidateList
def createCanvasCandidates(strokeLengthList, isVerticalList, sCoordinatesList, num, canvas, stroke, minL, maxL, columnNum, sideL):
    canvasList = []

    while(len(canvasList) < num):
        canvasCopy = canvas.copy()
        strokeCoordinates = []  # store the coordinates of the stroke

        # each stroke picks a random length in between the range
        currLength = random.randrange(minL, maxL)

        stroke1 = stroke.copy()
        # resize current stroke
        currStroke = stroke1.resize((currLength, stroke.size[1]))
        isVertical = False

        #if(tone <= 0.4):
            #placeStroke(currStroke, canvasCopy, sideL, strokeCoordinates)
        if(columnNum >= 4 and columnNum < 7):  # from certain point, only add vertical strokes
            currStroke = currStroke.rotate(rotateDegree, Image.NEAREST, expand = True)
            isVertical = True

        elif(columnNum >= 7):  # from certain point, add both horizontal and vertical strokes randomly
            choiceList = [True, False]
            rotation = random.choice(choiceList)

            # if expected direction and the currStroke's current direction are aligned - just place the stroke
            # if not aligned, then rotate to expected direction
            if(rotation == True and isVertical == False):  # currStroke is horizontal but we want vertical stroke
                currStroke = currStroke.rotate(rotateDegree, Image.NEAREST, expand = True)
                isVertical = True

        # all tone requirement need to apply the stroke
        placeStroke(currStroke, canvasCopy, sideL, strokeCoordinates)

        copyCanvTone = findCoverage(canvasCopy)
        maxToneBound = tone + toneBoundVal
        if(copyCanvTone > maxToneBound):  # if out of max Tone bound, do not add to candidate list, try another stroke
            continue
        else:  # if within max Tone bound, add to candidate list
            strokeLengthList.append(currLength)
            sCoordinatesList.append(strokeCoordinates)
            canvasList.append(canvasCopy)
            isVerticalList.append(isVertical)
    
    #print("candidate canvas prepared.")
    
    return canvasList

In [12]:
def findLiteralToneDiff(num, canvasList, canvas):
    toneDiffList = []

    for i in range(num):
        toneDiff = findCoverage(canvasList[i]) - findCoverage(canvas)
        #print(toneDiff)
        toneDiffList.append(toneDiff)
        
    #print("Direct tone difference found.")
    return toneDiffList

In [13]:
def findGaussianPyramids(count, canvasList):
    gPyramidList = []  # corresponding pyramids of each canvas candidate

    # create and store all Gaussian pyramid for each candidate canvas
    if(count != 0):
        candidateNum = len(canvasList)
        for i in range(candidateNum):
            currCan = canvasList[i]
            gPyramid = [] # the pyramid for the current canvas candidate

            for j in range(0, count):  # create Gaussian pyramid for current canvas candidate
                if(len(gPyramid) == 0):
                    gBlurImg = currCan
                else:
                    lastBlurInd = len(gPyramid) - 1
                    gBlurImg = gPyramid[lastBlurInd]

                gBlurImg = gBlurImg.filter(ImageFilter.GaussianBlur(2))  # Guassian Blur
                gBlurImg = gBlurImg.resize((int(gBlurImg.size[0]*0.5), int(gBlurImg.size[1]*0.5)))  # Shrink size
                #gBlurImg.show()
                gPyramid.append(gBlurImg)

                #gBlurImg.show()

            gPyramidList.append(gPyramid) # add current canvas candidate's pyramid into the pyramid list

    #print("gaussian pyramid for candidates created.")
    
    return gPyramidList

In [14]:
def findGaussianPyramidOriginCanv(canvas, count):
    canvasPyramid = []

    gBlurImg = canvas
    for i in range(count):
        gBlurImg = gBlurImg.filter(ImageFilter.GaussianBlur(2))  # Guassian Blur
        gBlurImg = gBlurImg.resize((int(gBlurImg.size[0]*0.5), int(gBlurImg.size[1]*0.5)))  # Shrink size
        #gBlurImg.show()
        canvasPyramid.append(gBlurImg)

        #gBlurImg.show()

    #print("gaussian pyramid for original canvas created.")
    #print(len(canvasList))
    
    return canvasPyramid

In [15]:
def findFinalToneDiff(canvasList, toneDiffList, strokeLengthList, count, gPyramidList, canvasPyramid):
    diffSumList = []  # store diff sum of each canvas candidate

    for i in range(len(canvasList)):
        # nomalization: only normalize the literal difference
        #literalDiff = toneDiffList[i] / strokeLengthList[i]
        literalDiff = toneDiffList[i]
        diffSumList.append(literalDiff)
        #print("LiteralDiff = " + str(literalDiff))
        # ******************************* Trail: Without literal diff, the texture distribution is worse ******************************************
        #diffSumList.append(0)

        #print("Literal Diff = " + str(toneDiffList[i]))
        # Gaussian pyramid Difference
        for j in range(count):
            # normalize also the pyramid difference by current strokeLength
            #pyramidDiff = (findCoverage(gPyramidList[i][j]) - findCoverage(canvasPyramid[j])) / (strokeLengthList[i] / pow(2, count))
            pyramidDiff = findCoverage(gPyramidList[i][j]) - findCoverage(canvasPyramid[j])
            diffSumList[i] += pyramidDiff
            #diffSumList[i] += findCoverage(gPyramidList[i][j]) - findCoverage(canvasPyramid[j])
            
    return diffSumList

In [16]:
def FindBestCanvInfo(diffSumList, canvasList, sCoordinatesList, strokeLengthList, isVerticalList, bestStrokePos, dummyStrokeLenList, dummyDirList):
    #largestDiff = diffSumList[0]/ (strokeLengthList[0]/100)
    #largestDiff = diffSumList[0]/ strokeLengthList[0]
    largestDiff = diffSumList[0]

    bestCanvas = canvasList[0]
    #bestStrokePos = sCoordinatesList[0]
    #dummyStrokeLenList[0] = strokeLengthList[0]
    #dummyDirList[0] = isVerticalList[0]
    for i in range(len(diffSumList)):  # the candidateCav with the largest diff is the best Candidate
        #*** normalize sumDiff with strokeLength ***
        #currDiff = diffSumList[i]/ (strokeLengthList[i]/100)
        #currDiff = diffSumList[i]/ strokeLengthList[i]
        currDiff = diffSumList[i]
        #print("in largest diff finding.")

        if(currDiff > largestDiff):
            bestCanvas = canvasList[i]
            #largestDiff = diffSumList[i]
            largestDiff = currDiff
            bestStrokePos[0] = sCoordinatesList[i][0]
            bestStrokePos[1] = sCoordinatesList[i][1]
            dummyStrokeLenList[0] = strokeLengthList[i]
            dummyDirList[0] = isVerticalList[i]

    print("Best canvas found.")
    return bestCanvas

In [17]:
columnNum = 1
totalToneNum = 16
#totalToneNum = 6
lastColumn = []

rotateDegree = 335

toneBoundVal = 0.02
candiNum = 1000
# ^^^^^^^^^^^^^^^^ after generated one column, move to next column (darker tone): repeat from here^^^^^^^^^^^^^^^^^^^^^^^^
while (columnNum <= totalToneNum):
    # create all the texture canvas in current column and store in a list
    if(columnNum == 1):  # if first column: empty canvas
        sideL = 16
        tCanvasList = []  # store all texture canvas
        while (sideL < 512):
            tempCanv = Image.new("RGB", (sideL,sideL),(255,255,255))
            tCanvasList.append(tempCanv)
            sideL *= 2
    
    tone = 0.0
    # choose corresponding tone for this current column
    # using only gamma correction

    """
    # try 6 columns
    textureNum = 6
    interval = math.pi / (2 * (6 + 1))
    gammaCoefficient = 2.2
    
    if(columnNum == 1):
        lightTone = math.cos(interval * 6)
        tone = pow(lightTone, 1/gammaCoefficient)
    if(columnNum == 2):
        lightTone = math.cos(interval * 5)
        tone = pow(lightTone, 1/gammaCoefficient)
    if(columnNum == 3):
        lightTone = math.cos(interval * 4)
        tone = pow(lightTone, 1/gammaCoefficient)
    if(columnNum == 4):
        lightTone = math.cos(interval * 3)
        tone = pow(lightTone, 1/gammaCoefficient)
    if(columnNum == 5):
        lightTone = math.cos(interval * 2)
        tone = pow(lightTone, 1/gammaCoefficient)
    if(columnNum == 6):
        lightTone = math.cos(interval * 1)
        tone = pow(lightTone, 1/gammaCoefficient)
    """
    
    
    # try 6 columns with original tone value from Praun et al. 
    textureNum = 6
    """
    if(columnNum == 1):
        tone = 0.430
    if(columnNum == 2):
        tone = 0.724
    if(columnNum == 3):
        tone = 0.875
    if(columnNum == 4):
        tone = 0.946
    if(columnNum == 5):
        tone = 0.988
    if(columnNum == 6):
        tone = 0.999
    """
    # try 6 columns with linear tone value 
    """
    if(columnNum == 1):
        tone = 0.2
    if(columnNum == 2):
        tone = 0.3
    if(columnNum == 3):
        tone = 0.4
    if(columnNum == 4):
        tone = 0.5
    if(columnNum == 5):
        tone = 0.6
    if(columnNum == 6):
        tone = 0.7
    if(columnNum == 7):
        tone = 0.8
    if(columnNum == 8):
        tone = 0.9
    """
    # current project uses 16 columns with gamma corrected tone value 
    if(columnNum == 1):
        tone = 0.176
    if(columnNum == 2):
        tone = 0.32
    if(columnNum == 3):
        tone = 0.464
    if(columnNum == 4):
        tone = 0.584
    if(columnNum == 5):
        tone = 0.696
    if(columnNum == 6):
        tone = 0.7696
    if(columnNum == 7):
        tone = 0.828
    if(columnNum == 8):
        tone = 0.912
    if(columnNum == 9):
        tone = 0.94
    if(columnNum == 10):
        tone = 0.958
    if(columnNum == 11):
        tone = 0.976
    if(columnNum == 12):
        tone = 0.984
    if(columnNum == 13):
        tone = 0.987701416
    if(columnNum == 14):
        tone = 0.99362945
    if(columnNum == 15):
        tone = 0.999557495
    if(columnNum == 16):
        tone = 1.0
    
    
    print(columnNum)
    print(tone)
        
    # ^^^^^^^^^^^^^if current texture finished, move to the next larger one in the same column: repeat from here^^^^^^^^^^^^^^^^^^^^^^^^
    # generate one column of texture
    for mipLevel in range(5):
        canvas = tCanvasList[mipLevel]
        sideL = canvas.size[0]       
        #canvas.show()

        # set appropriate stroke length range (0.4 - 0.9 of the texture width) for this texture size
        stroke = Image.open(r"D:\UWGraduate\GraduateProject\TAM\TAM_PS\MyStroke.png")
        #stroke.show()

        minL = round(sideL*0.4)
        maxL = round(sideL*0.95)
        #print(minL, maxL)
        
        # ^^^^^^^^^^^^^^^^^^^apply stroke (if general tone has not been reached, we repeat from here)^^^^^^^^^^^^^^^^^^^
        while True:
            # to indicate if this current stroke is vertical or not
            isVertical = False
            
            # try multiple candidate stroke placement of current stroke, pick the best one
            # randomly place strokes onto the canvas
            num = 0 # number of candidate stroke
            if (columnNum == 1):
                num = candiNum
            elif (columnNum == 2):
                num = candiNum
            elif (columnNum == 3):
                num = candiNum
            elif (columnNum == 4):
                num = candiNum
            elif (columnNum == 5):
                num = candiNum
            elif (columnNum == 6):
                num = candiNum
            elif (columnNum == 7):
                num = candiNum
            elif (columnNum == 8):
                num = candiNum
            elif (columnNum == 9):
                num = candiNum
            elif (columnNum == 10):
                num = candiNum
            elif (columnNum == 11):
                num = candiNum
            elif (columnNum == 12):
                num = candiNum
            elif (columnNum == 13):
                num = candiNum
            elif (columnNum == 14):
                num = candiNum
            elif (columnNum == 15):
                num = candiNum
            elif (columnNum == 16):
                num = candiNum



            # Create all canvas candidates in the list
            strokeLengthList = []  # store the length of each candidate stroke
            isVerticalList = []    # store current candidate's direction (final tons have random direction, need this info for passing down the strokes)
            sCoordinatesList = []  # store the coordinates of current stroke position in corresponding list
            canvasList = createCanvasCandidates(strokeLengthList, isVerticalList, sCoordinatesList, num, canvas, stroke, minL, maxL, columnNum, sideL) # store all candidate canvas with a randomly candidate stroke in the list
            
            # FIND BEST CANVAS
            # Step 1: find Literal tone difference
            # store the coverage difference between the new canvas with currStroke and prev canvas without currStroke
            toneDiffList = findLiteralToneDiff(num, canvasList, canvas)

            # Step 2: Gussian Pyramid
            count = 0  # decide Gussian pyramid level - processing times
            if(sideL == 16):
                count = 1
            if(sideL == 32):
                count = 2
            if(sideL == 64):
                count = 3
            if(sideL == 128):
                count = 4
            if(sideL == 256):
                count = 5

            gPyramidList = findGaussianPyramids(count, canvasList)
            
            # create gaussian pyramid of the current canvas
            #if(sideL != 16):  # Alternative trial: if 16x16, no need for gaussian pyramid processing
            canvasPyramid = findGaussianPyramidOriginCanv(canvas, count)
            
            
            # Step 3: calculate final tone differences
            if(len(toneDiffList) == 0):
                print("Error: toneDiffList is empty")
            else:
                """ # Alternative trial: if 16x16, no need for gaussian pyramid processing
                largestDiff = toneDiffList[0]
                if(sideL == 16):  # if texture size is 16x16, no gaussian pyramid diff
                    for i in range(len(toneDiffList)):  # the candidateCav with the largest diff is the best Candidate
                        if(toneDiffList[i] > largestDiff):
                            bestCanvas = canvasList[i]
                            largestDiff = toneDiffList[i]
                            bestStrokePos = sCoordinatesList[i]
                """
                diffSumList = findFinalToneDiff(canvasList, toneDiffList, strokeLengthList, count, gPyramidList, canvasPyramid)
                
                # Step 4: Find best canvas
                bestCanvas = canvasList[0]
                bestStrokePos = []
                bestStrokePos.append(sCoordinatesList[0][0])
                bestStrokePos.append(sCoordinatesList[0][1])
                dummyStrokeLenList = []
                dummyStrokeLenList.append(strokeLengthList[0])
                dummyDirList = []
                dummyDirList.append(isVerticalList[0])
                
                bestCanvas = FindBestCanvInfo(diffSumList, canvasList, sCoordinatesList, strokeLengthList, isVerticalList, bestStrokePos, dummyStrokeLenList, dummyDirList)
                
                bestStrokeLen = dummyStrokeLenList[0]
                bestCanvDir = dummyDirList[0]
            
            # check if the general tone has reached the predefined level
            generalTone = findCoverage(bestCanvas)
            minToneBound = tone - toneBoundVal
            maxToneBound = tone + toneBoundVal
            if (generalTone >= minToneBound and generalTone <= maxToneBound):  # reached predefined tone - save the texture
                # get current applied stroke's length and relative position, then apply to the rest texture canvas in the same column
                # keep all strokes in small texture also present in larger texture
                # currStroke: the current stroke pasted; bestStrokePos( = sCoordinatesList[i]): relative position percentage of the current stroke's position
                currLevel = mipLevel + 1
                while (currLevel < 5):
                    # add currStroke to all larger level of canvas at the same relative position, with the same realtive length
                    longStroke = resizeStrokeForNextLevel(stroke, bestStrokeLen, bestCanvDir, bestCanvas, tCanvasList[currLevel])  # resize the currStroke to relative same length
                    placeStrokeWithPos(longStroke, tCanvasList[currLevel], bestStrokePos)
                    currLevel += 1
                    
                #print("new stroke added to all canvas in the same column.")
                
                # update the current canvas in tCanvasList (current working column) (canvas not yet updated)
                tCanvasList[mipLevel] = bestCanvas
                
                #bestCanvas.show()  # this is the canvas with the best candidate stroke
                canvasName = str(sideL) + "Tone" + str(tone) + ".png"
                canvasName = "D:/UWGraduate/GraduateProject/TAM/MyGenerator/MyTAM/" + canvasName
                bestCanvas.save(canvasName)

                print("Completed: " + str(sideL) + "Tone" + str(tone))
                
                break
            elif (generalTone < minToneBound):  # have not reached the defined tone yet - continue adding the next stroke
                # get current applied stroke's length and relative position, then apply to the rest texture canvas in the same column
                # keep all strokes in small texture also present in larger texture
                # currStroke: the current stroke pasted; bestStrokePos( = sCoordinatesList[i]): relative position percentage of the current stroke's position
                currLevel = mipLevel + 1
                while (currLevel < 5):
                    # add currStroke to all larger level of canvas at the same relative position, with the same realtive length
                    longStroke = resizeStrokeForNextLevel(stroke, bestStrokeLen, bestCanvDir, bestCanvas, tCanvasList[currLevel])  # resize the currStroke to relative same length
                    placeStrokeWithPos(longStroke, tCanvasList[currLevel], bestStrokePos)
                    currLevel += 1

                #print("new stroke added to all canvas in the same column.")
                
                canvas = bestCanvas  # canvas take the current best candidate

                #print("Added a stroke for: " + str(sideL) + "Tone" + str(tone))
                
                continue
            else:  # beyond the defined tone
                print("!!!!!!!!!!!!!!!current tone = " + str(generalTone))
                print("CurrStroke too long.")
                continue   # canvas is the original without the currStroke in this iteration
                            # start with the original canvas again
                            # no passing down currentStroke to the larger textures
        
    # ****** if out of the for loop: the current column is finished, need to move to the next column ******
    columnNum += 1  # increment
    
# *********************** Memory consumption should be okay*********************************
# *********************** Memory is cheap now & every loop we reallocate new memory at the same place *******************

1
0.176
Best canvas found.
Completed: 16Tone0.176
Best canvas found.
Completed: 32Tone0.176
Best canvas found.
Best canvas found.
Completed: 64Tone0.176
Best canvas found.
Best canvas found.
Best canvas found.
Completed: 128Tone0.176
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Completed: 256Tone0.176
2
0.32
Best canvas found.
Completed: 16Tone0.32
Best canvas found.
Completed: 32Tone0.32
Best canvas found.
Best canvas found.
Best canvas found.
Completed: 64Tone0.32
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Completed: 128Tone0.32
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Best canvas found.
Completed: 256Tone0.32
3
0.464
Best canvas found.
Completed: 16Tone0.464
Best canvas found.
Completed: 32Tone0.464
Best canvas found.
Best canvas found.
Completed: 64Tone0.464
Best canvas found.
Best canvas fo

In [11]:
# read in original and find its tone
tex1 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\OrigianlTexture\hatch_0.jpg")
tex2 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\OrigianlTexture\hatch_1.jpg")
tex3 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\OrigianlTexture\hatch_2.jpg")
tex4 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\OrigianlTexture\hatch_3.jpg")
tex5 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\OrigianlTexture\hatch_4.jpg")
tex6 = Image.open(r"D:\UWGraduate\GraduateProject\TAM\MyGenerator\MyTAM\OrigianlTexture\hatch_5.jpg")


testTone1 = findCoverage(tex1)
testTone2 = findCoverage(tex2)
testTone3 = findCoverage(tex3)
testTone4 = findCoverage(tex4)
testTone5 = findCoverage(tex5)
testTone6 = findCoverage(tex6)

print(testTone1)
print(testTone2)
print(testTone3)
print(testTone4)
print(testTone5)
print(testTone6)

0.429901123046875
0.7243499755859375
0.8753814697265625
0.94598388671875
0.987701416015625
0.9995574951171875


In [ ]:
# reference for gamma correction
# https://www.cambridgeincolour.com/tutorials/gamma-correction.htm#:~:text=Gamma%20correction%20is%20sometimes%20specified,1%2F1.5%20%3D%201.0).
# https://www.dfstudios.co.uk/articles/programming/image-programming-algorithms/image-processing-algorithms-part-6-gamma-correction/